# ISO TC/211 Harmonised Model Dependency Controls

Connect to the EA app and model repository

In [3]:
from Parameters import *
from EAConnect import *
from HM_Controls import *
import sys
import pandas as pd

# Open EA Repository and find Model
eaApp = openEAapp()
eaRepo = openEArepo(eaApp,repo_path)
try:
    cmMod = eaRepo.Models.GetByName(modelName)
    printTS('Model "' + modelName + '" found with PackageGUID ' + cmMod.PackageGUID )
    printTS('Number of main packages: ' + str(cmMod.Packages.Count))
except Exception as e:
    printTS('Model  "' + modelName + '" not found!')
    # closeEA(eaRepo)
    # sys.exit()

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

2024-07-01 10:46:47   Hi EA - are you there? 
2024-07-01 10:46:47   I am here
2024-07-01 10:46:47   Hi EA - Please open this repository: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\ISOTC211_HM EditorialVersion.qea
2024-07-01 10:46:48   OK! Repository C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\ISOTC211_HM EditorialVersion.qea is ready!
2024-07-01 10:46:48   Model "Conceptual Models" found with PackageGUID {7B6B28E9-C583-4363-9E9C-F37A37AE06C9}
2024-07-01 10:46:48   Number of main packages: 5


Get the TC 211 main package

In [4]:
tcmName = 'ISO TC211'
try:
    tcMod = cmMod.Packages.GetByName(tcmName)
    printTS('Model "' + tcmName + '" found with PackageGUID ' + tcMod.PackageGUID )
except Exception as e:
    printTS('Model  "' + tcmName + '" not found!')

printTS('Number of main packages: ' + str(tcMod.Packages.Count))

2024-07-01 10:46:52   Model "ISO TC211" found with PackageGUID {CAB2E56D-50FA-4904-A16C-B34D7AE325B6}
2024-07-01 10:46:52   Number of main packages: 58


 # Create DataFrames for elements and properties
 - Loop through the complete model:
    - For each element: add content to rows for ElementID, GUID, main standard, standard and element name
    - For each attribute or association end: add content to rows for elementID, GUID, main standard, standard, attribute/association name and refElementID

In [42]:
def recElements(pck,df):
    for eaEl in pck.Elements:
        # print('Element: ' + el.Name)
        if eaEl.Type.upper() in ["CLASS","INTERFACE", "DATATYPE","ENUMERATION"]:
           df.loc[len(df)] = [eaEl.ElementID, eaEl.ElementGUID, stPck.PackageGUID, stPck.Name,edPck.PackageGUID, edPck.Name,eaEl.Name,eaEl.Type] 

    for p in pck.Packages:
        recElements(p,df)
    return df

dfEl = pd.DataFrame(columns=['ElementID','GUID','msGUID','MainStandard','edGUID', 'Edition', 'ElementName','Type'])
for stPck in tcMod.Packages:
    printTS('Standard: ' + stPck.Name)
    for edPck in stPck.Packages:
        printTS('Edition: ' + edPck.Name)
        dfEl = recElements(edPck,dfEl)


2024-07-01 13:22:21   Standard: Common types
2024-07-01 13:22:21   Standard: Package Context Diagrams
2024-07-01 13:22:21   Standard: ISO 6709 Standard representation of geographic point location by coordinates
2024-07-01 13:22:21   Edition: ISO 6709 Edition 2
2024-07-01 13:22:21   Edition: ISO 6709 Edition 1 (Corrigendum 1)
2024-07-01 13:22:21   Standard: ISO 19101 Reference model
2024-07-01 13:22:21   Edition: ISO 19101-1 Edition 1
2024-07-01 13:22:21   Edition: ISO 19101-2 Edition 1
2024-07-01 13:22:21   Edition: ISO/TS 19101-2 Edition 1
2024-07-01 13:22:21   Standard: ISO 19103 Conceptual schema language
2024-07-01 13:22:21   Edition: ISO/FDIS 19103 Edition 2
2024-07-01 13:22:22   Edition: ISO/FDIS 19103 Edition 2 Informative content
2024-07-01 13:22:22   Edition: ISO 19103 Edition 1
2024-07-01 13:22:23   Edition: ISO TS 19103 Edition 1
2024-07-01 13:22:24   Standard: ISO 19104 Terminology
2024-07-01 13:22:24   Edition: ISO 19104 Edition 1
2024-07-01 13:22:24   Standard: ISO 19105 

In [20]:
def recProperties(pck,df):
    for eaEl in pck.Elements:
        # print('Element: ' + el.Name)
        if eaEl.Type.upper() in ["CLASS","INTERFACE", "DATATYPE","ENUMERATION"]:
            for eaProp in eaEl.Attributes:
                df.loc[len(df)] = [eaEl.ElementID, eaEl.ElementGUID, stPck.PackageGUID, stPck.Name,edPck.PackageGUID,edPck.Name, eaProp.Name,'Attribute',eaProp.ClassifierID] 
            # TODO: Navigable assocation ends
             #Loop for connector dependencies
            for eaCon in eaEl.Connectors:
                if eaCon.SupplierID == eaEl.ElementID:
                    cEnd = eaCon.ClientEnd
                    ClassifierID = eaCon.ClientID 
                else:
                    cEnd = eaCon.SupplierEnd
                    ClassifierID = eaCon.SupplierID

                df.loc[len(df)] = [eaEl.ElementID, eaEl.ElementGUID, stPck.PackageGUID, stPck.Name,edPck.PackageGUID,edPck.Name, cEnd.Role,eaCon.Type,ClassifierID] 

    for p in pck.Packages:
        recProperties(p,df)
    return df

dfProp = pd.DataFrame(columns=['ElementID','GUID','msGUID','MainStandard','edGUID', 'Edition', 'PropertyName','Type','refElementID'])
for stPck in tcMod.Packages:
    printTS('Standard: ' + stPck.Name)
    for edPck in stPck.Packages:
        printTS('Edition: ' + edPck.Name)
        dfProp = recProperties(edPck,dfProp)

2024-07-01 12:30:00   Standard: Common types
2024-07-01 12:30:00   Standard: Package Context Diagrams
2024-07-01 12:30:00   Standard: ISO 6709 Standard representation of geographic point location by coordinates
2024-07-01 12:30:00   Edition: ISO 6709 Edition 2
2024-07-01 12:30:00   Edition: ISO 6709 Edition 1 (Corrigendum 1)
2024-07-01 12:30:00   Standard: ISO 19101 Reference model
2024-07-01 12:30:00   Edition: ISO 19101-1 Edition 1
2024-07-01 12:30:01   Edition: ISO 19101-2 Edition 1
2024-07-01 12:30:01   Edition: ISO/TS 19101-2 Edition 1
2024-07-01 12:30:02   Standard: ISO 19103 Conceptual schema language
2024-07-01 12:30:02   Edition: ISO/FDIS 19103 Edition 2
2024-07-01 12:30:02   Edition: ISO/FDIS 19103 Edition 2 Informative content
2024-07-01 12:30:03   Edition: ISO 19103 Edition 1
2024-07-01 12:30:05   Edition: ISO TS 19103 Edition 1
2024-07-01 12:30:07   Standard: ISO 19104 Terminology
2024-07-01 12:30:07   Edition: ISO 19104 Edition 1
2024-07-01 12:30:07   Standard: ISO 19105 

In [17]:
print(dfProp.head(10))

   ElementID                                    GUID  \
0          9  {375618E4-18C3-46fb-898E-E141DA840492}   
1          9  {375618E4-18C3-46fb-898E-E141DA840492}   
2         10  {A57913B0-B61D-4de4-968A-89771001D198}   
3        118  {26610375-01C5-44f1-9B56-2A25049AEDEA}   
4        118  {26610375-01C5-44f1-9B56-2A25049AEDEA}   
5        116  {2541775F-9070-49de-AB98-28F0329A8C6D}   
6        119  {4A7D91D7-6A2E-4e73-9BE8-9C6691F27D0E}   
7        121  {D7271D77-49A8-43b2-A5BE-1F731A217309}   
8        124  {3F935969-2134-4f0a-8ADA-38DF62AD1D18}   
9        124  {3F935969-2134-4f0a-8ADA-38DF62AD1D18}   

                                   msGUID  \
0  {44346D64-FF7D-40bb-B945-570B5E243B5C}   
1  {44346D64-FF7D-40bb-B945-570B5E243B5C}   
2  {44346D64-FF7D-40bb-B945-570B5E243B5C}   
3  {156E812E-1496-4aa2-8819-399111907257}   
4  {156E812E-1496-4aa2-8819-399111907257}   
5  {156E812E-1496-4aa2-8819-399111907257}   
6  {156E812E-1496-4aa2-8819-399111907257}   
7  {156E812E-1496-4aa2-

# Count references
- per element
- per element and edition
- per element and main standard

In [54]:
# Total number of references to each element

dfElCounts = dfProp.groupby(['refElementID'])['ElementID'].count().reset_index()
dfElCounts.rename(columns={'ElementID': 'TotalReferences'}, inplace=True)
# print(dfElCounts)

result_df = pd.merge(dfEl, dfElCounts, left_on='ElementID', right_on='refElementID', how='left')
result_df.drop(columns=['refElementID'], inplace=True)
result_df.fillna({'TotalReferences':0},inplace=True)
result_df['TotalReferences'] = result_df['TotalReferences'].astype(int)
dfElStat = result_df

# Total number references to per edition package to each element
#Group by edGUID and refElement
dfElCounts = dfProp.groupby(['refElementID','edGUID'])['ElementID'].count().reset_index()
dfElCounts.rename(columns={'ElementID': 'EditionInternalReferences'}, inplace=True)
#Merge on edGUID and refElement
result_df = pd.merge(dfElStat, dfElCounts, left_on=['ElementID','edGUID'], right_on=['refElementID','edGUID'], how='left')
result_df.drop(columns=['refElementID'], inplace=True)
result_df.fillna({'EditionInternalReferences':0},inplace=True)
result_df['EditionInternalReferences'] = result_df['EditionInternalReferences'].astype(int)
dfElStat = result_df

# Total number references to per main standards package to each element
#Group by msGUID and refElement
dfElCounts = dfProp.groupby(['refElementID','msGUID'])['ElementID'].count().reset_index()
dfElCounts.rename(columns={'ElementID': 'StandardInternalReferences'}, inplace=True)
#Merge on edGUID and refElement
result_df = pd.merge(dfElStat, dfElCounts, left_on=['ElementID','msGUID'], right_on=['refElementID','msGUID'], how='left')
result_df.drop(columns=['refElementID'], inplace=True)
result_df.fillna({'StandardInternalReferences':0},inplace=True)
result_df['StandardInternalReferences'] = result_df['StandardInternalReferences'].astype(int)
dfElStat = result_df
dfElStat['ExternalReferences'] = dfElStat['TotalReferences'] - dfElStat['StandardInternalReferences']
# dfElStat.drop(columns=['StandardInternalReferences','EditionInternalReferences'], inplace=True)


In [58]:
# Statistics per element name per main standard (to summarize all references for different versions of the same element)
elStatByName = dfElStat.groupby(['msGUID', 'MainStandard','ElementName']).agg(
    VersionCount=pd.NamedAgg(column='ElementID', aggfunc='count'),
    ExternalReferences=pd.NamedAgg(column='ExternalReferences', aggfunc='sum'),
    TotalReferences=pd.NamedAgg(column='TotalReferences', aggfunc='sum')
).reset_index()



# Export to Excel

In [61]:
print('')
file_path = mainFolder + '\\Dependencies.xlsx'
# Export to Excel 
writer = pd.ExcelWriter(file_path)
dfElStat.to_excel(writer,'Elements') 
print(f"Exported elements report to file: {file_path}") 

elStatByName.to_excel(writer,'Elements by standard') 
print(f"Exported elements report to file: {file_path}") 

dfProp.to_excel(writer,'Properties') 
print(f"Exported properties report to file: {file_path}") 

writer.close()


C:\Users\JETKNU\AppData\Local\Temp\ipykernel_17708\2208172230.py:5: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  dfElStat.to_excel(writer,'Elements')


Exported elements report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Dependencies.xlsx


C:\Users\JETKNU\AppData\Local\Temp\ipykernel_17708\2208172230.py:8: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  elStatByName.to_excel(writer,'Elements by standard')


Exported elements report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Dependencies.xlsx


C:\Users\JETKNU\AppData\Local\Temp\ipykernel_17708\2208172230.py:11: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  dfProp.to_excel(writer,'Properties')


Exported properties report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Dependencies.xlsx


In [ ]:
# List properties refering to a specific element
